# ocurl_example

[ocurl](http://ygrek.org.ua/p/ocurl/) is a binding to libcurl, a popular easy-to-use HTTP(S) client library.
This library is useful to access to API servers or Web scraping.

In [1]:
#require "core" ;;
#require "curl" ;;
#require "uri" ;;
#require "yojson" ;;

/home/opam/.opam/4.04.1/lib/base/caml: added to search path
/home/opam/.opam/4.04.1/lib/base/caml/caml.cma: loaded
/home/opam/.opam/4.04.1/lib/base/shadow_stdlib: added to search path
/home/opam/.opam/4.04.1/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/home/opam/.opam/4.04.1/lib/sexplib/0: added to search path
/home/opam/.opam/4.04.1/lib/sexplib/0/sexplib0.cma: loaded
/home/opam/.opam/4.04.1/lib/base: added to search path
/home/opam/.opam/4.04.1/lib/base/base.cma: loaded
/home/opam/.opam/4.04.1/lib/ocaml/unix.cma: loaded
/home/opam/.opam/4.04.1/lib/ocaml/bigarray.cma: loaded
/home/opam/.opam/4.04.1/lib/fieldslib: added to search path
/home/opam/.opam/4.04.1/lib/fieldslib/fieldslib.cma: loaded
/home/opam/.opam/4.04.1/lib/ppx_compare/runtime-lib: added to search path
/home/opam/.opam/4.04.1/lib/ppx_compare/runtime-lib/ppx_compare_lib.cma: loaded
/home/opam/.opam/4.04.1/lib/sexplib: added to search path
/home/opam/.opam/4.04.1/lib/sexplib/sexplib.cma: loaded
/home/opam/.opam/4.04.1/l

In [2]:
open Core ;;

In [3]:
Curl.global_init Curl.CURLINIT_GLOBALALL ;;

- : unit = ()

In [4]:
let search_query query =
  let b = Buffer.create 128 in
  let write_result s =
    Buffer.add_string b s ;
    String.length s
  in
  let base_uri = Uri.of_string "http://api.duckduckgo.com/?format=json" in
  let uri = Uri.add_query_param base_uri ("q", [query]) |> Uri.to_string in
  let c = Curl.init () in
  Curl.set_writefunction c write_result ;
  Curl.set_followlocation c true ;
  Curl.set_url c uri ;
  Curl.set_sslverifypeer c true ;
  Curl.perform c ;
  Curl.cleanup c ;
  let body = Buffer.contents b in
  let json = Yojson.Basic.from_string body in (* Parse a JSON *)
  (* Find field "Definition" or "Abstract" in a JSON *)
  let def = Yojson.Basic.Util.(json |> member "Definition" |> to_string_option) in
  let abs = Yojson.Basic.Util.(json |> member "Abstract" |> to_string_option) in
  match def with
  | None | Some "" -> abs
  | _ -> def

val search_query : string -> string option = <fun>

In [5]:
search_query "ocaml"

- : string option =
Some
 "OCaml, originally named Objective Caml, is the main implementation of the programming language Caml, created by Xavier Leroy, J\195\169r\195\180me Vouillon, Damien Doligez, Didier R\195\169my, Asc\195\161nder Su\195\161rez and others in 1996. A member of the ML language family, OCaml extends the core Caml language with object-oriented programming constructs."